In [17]:
wine=read.csv("wine.csv",sep=",")
wine[,2]

[1] 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 [38] 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 [75] 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
[112] 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
[149] 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3

In [18]:
X=wine[,c(-1,-2)]
y=wine[,2]
for(i in 1:ncol(X)){
  X[is.na(X[,i]), i] <- mean(X[,i], na.rm = TRUE)
}


In [19]:
library(RSNNS)

#split train test
split.wine<-splitForTrainingAndTest(X,y,ratio=0.10)
# $inputsTrain $targetsTrain $inputsTest $targetsTest
split.wine$targetsTest

[1] 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3

In [20]:
library("e1071")

svm_model <- svm(split.wine$inputsTrain,split.wine$targetsTrain,kernel="radial",gamma=0.03,type="C-classification")
summary(svm_model)
predicted <- predict(svm_model,split.wine$inputsTest)
#?svm


Call:
svm.default(x = split.wine$inputsTrain, y = split.wine$targetsTrain, 
    type = "C-classification", kernel = "radial", gamma = 0.03)


Parameters:
   SVM-Type:  C-classification 
 SVM-Kernel:  radial 
       cost:  1 
      gamma:  0.03 

Number of Support Vectors:  65

 ( 18 32 15 )


Number of Classes:  3 

Levels: 
 1 2 3




In [21]:
table(predicted,split.wine$targetsTest)
#predicted

         
predicted  3
        1  0
        2  0
        3 18

In [22]:
###set max iterations T
T=5
#initialize sample weights
dm=replicate(nrow(split.wine$inputsTrain),1/nrow(split.wine$inputsTrain))
length(dm)

[1] 160

In [23]:
library("wSVM")
#svm.wt=wsvm(split.wine$inputsTrain, split.wine$targetsTrain, c.n=dm, kernel = list(type = 'rbf', par = NULL))

?wsvm

In [24]:
library(gmum.r)
library(SparseM)
library(Matrix)
library(ggplot2)
        
svm.wt <- SVM(split.wine$inputsTrain,split.wine$targetsTrain, core="svmlight", kernel="rbf", C=1.0, 
                        gamma=0.5, example.weights=dm)

#summary(svm.wt)
predicted <- predict(svm.wt,split.wine$inputsTrain)
as.numeric(predicted)

Warning message in SVM.default(split.wine$inputsTrain, split.wine$targetsTrain, :
“It is recommended to pass y as factor”

[1] 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 [38] 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 [75] 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
[112] 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 3 3 3 2 3 3 3 3 2 2 2 2 3 3 3 3 3
[149] 3 3 3 3 2 3 2 3 3 3 2 3

In [25]:
###aplha parameter from adaboost algorith 
alpha=c()

###list to store predictions
pred=list()

###list to store test vector predictions
testpred=list()

###define number of classes
classes=3
dm

[1] 0.00625 0.00625 0.00625 0.00625 0.00625 0.00625 0.00625 0.00625 0.00625
 [10] 0.00625 0.00625 0.00625 0.00625 0.00625 0.00625 0.00625 0.00625 0.00625
 [19] 0.00625 0.00625 0.00625 0.00625 0.00625 0.00625 0.00625 0.00625 0.00625
 [28] 0.00625 0.00625 0.00625 0.00625 0.00625 0.00625 0.00625 0.00625 0.00625
 [37] 0.00625 0.00625 0.00625 0.00625 0.00625 0.00625 0.00625 0.00625 0.00625
 [46] 0.00625 0.00625 0.00625 0.00625 0.00625 0.00625 0.00625 0.00625 0.00625
 [55] 0.00625 0.00625 0.00625 0.00625 0.00625 0.00625 0.00625 0.00625 0.00625
 [64] 0.00625 0.00625 0.00625 0.00625 0.00625 0.00625 0.00625 0.00625 0.00625
 [73] 0.00625 0.00625 0.00625 0.00625 0.00625 0.00625 0.00625 0.00625 0.00625
 [82] 0.00625 0.00625 0.00625 0.00625 0.00625 0.00625 0.00625 0.00625 0.00625
 [91] 0.00625 0.00625 0.00625 0.00625 0.00625 0.00625 0.00625 0.00625 0.00625
[100] 0.00625 0.00625 0.00625 0.00625 0.00625 0.00625 0.00625 0.00625 0.00625
[109] 0.00625 0.00625 0.00625 0.00625 0.00625 0.00625 0.00625 0.00625 0.00625
[118] 0.00625 0.00625 0.00625 0.00625 0.00625 0.00625 0.00625 0.00625 0.00625
[127] 0.00625 0.00625 0.00625 0.00625 0.00625 0.00625 0.00625 0.00625 0.00625
[136] 0.00625 0.00625 0.00625 0.00625 0.00625 0.00625 0.00625 0.00625 0.00625
[145] 0.00625 0.00625 0.00625 0.00625 0.00625 0.00625 0.00625 0.00625 0.00625
[154] 0.00625 0.00625 0.00625 0.00625 0.00625 0.00625 0.00625

In [26]:

###run adabosst multiclass with cost vector to each class
for (t in 1:T){
svm.wt <- SVM(split.wine$inputsTrain,split.wine$targetsTrain, core="svmlight", kernel="rbf", C=1.0, 
                        gamma=0.5, example.weights=dm)

#summary(svm.wt)
#predict test values
predicted_test <- predict(svm.wt,split.wine$inputsTest)
#predict train values
predicted_train<- predict(svm.wt,split.wine$inputsTrain)    

    
#store predicted test values
testpred[[t]]<-(predicted_test)
    
#store predicted train values
pred[[t]]<-(predicted_train)
    
#initially for each iteration error==0
err=0
    
    #for each example in train predictions check misclassification
    for (i in (1:nrow(split.wine$inputsTrain))){
        if (pred[[t]][i] != split.wine$targetsTrain[i]){
            y=split.wine$targetsTrain[i]
            err=err + dm[i] #* cost[y-1]
            }
        }
    #calculate alpha wth err value and classes
    alpha[t]<-(log((1-err)/err) + log(classes-1))
    
    #calculate Z value for normaliztion
    Z=c()
    #print(err)
    for (i in (1:nrow(split.wine$inputsTrain))){
        #print(dm)
        #print(as.numeric( predicted_train[i]))
        Z[i]<-(dm[i] * exp((-(alpha[t]))*(split.wine$targetsTrain[i]) *as.numeric( predicted_train[i])))
        #print(Z)
    }
    #update the weights
    
    for (i in (1:nrow(split.wine$inputsTrain))){
        dm[i]=Z[i] / sum(Z)
        #print(dm)
        }

        
}   


Warning message in SVM.default(split.wine$inputsTrain, split.wine$targetsTrain, :
“It is recommended to pass y as factor”Warning message in SVM.default(split.wine$inputsTrain, split.wine$targetsTrain, :
“It is recommended to pass y as factor”Warning message in SVM.default(split.wine$inputsTrain, split.wine$targetsTrain, :
“It is recommended to pass y as factor”Warning message in SVM.default(split.wine$inputsTrain, split.wine$targetsTrain, :
“It is recommended to pass y as factor”Warning message in SVM.default(split.wine$inputsTrain, split.wine$targetsTrain, :
“It is recommended to pass y as factor”

In [27]:
#initialize the adaboost ensemble otput for each class
ada_class=c()
ada_class_pred=list()
#run for loop for number of classes
for (k in (1:classes)){
    
    #initialize predictions for each test vector
    
    
    #for each test vector 
    #for (i in 1:nrow(testpred[t]) ){
        
        #initialize the adaboost output as zero for each class
        ada_predicts=replicate(length(testpred[[1]]),0)
        #calculate the adaboost predicted value across t generations.this is a scalar
        for (t in 1:T){
            vec=testpred[[t]]
            al=alpha[t]
            for (i in 1:length(vec)){
            if (vec[i]==k){
                ada_predicts[i]= ada_predicts[i]+(al *1)
                }
            else{
                ada_predicts[i]=ada_predicts[i]+(al *0)
                }
            }
        }
        
    ada_class_pred[[k]]<-(ada_predicts)
        
    #}
    #predictions of each test vector across classes
    #ada_class.append(ada_class_pred)
}

In [28]:
x<-ada_class_pred
x

[[1]]
 [1] 165.7133 165.7133 165.7133 165.7133 165.7133 165.7133 165.7133 165.7133
 [9] 165.7133 165.7133 165.7133 165.7133 165.7133 165.7133 165.7133 165.7133
[17] 165.7133 165.7133

[[2]]
 [1] 0.000000 0.000000 3.513202 0.000000 0.000000 0.000000 0.000000 0.000000
 [9] 0.000000 0.000000 3.513202 0.000000 0.000000 0.000000 0.000000 0.000000
[17] 0.000000 0.000000

[[3]]
 [1] 3.513202 3.513202 0.000000 3.513202 3.513202 3.513202 3.513202 3.513202
 [9] 3.513202 3.513202 0.000000 3.513202 3.513202 3.513202 3.513202 3.513202
[17] 3.513202 3.513202

In [15]:
x<-ada_class_pred
cl=c()
for (i in 1:length(testpred[[1]]))
{
m = as.numeric(do.call( rbind, x)[,i])
cl[i]<-which(m==max(m))
}
cl

[1] 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1

In [14]:
acc = length(which(split.wine$targetsTest == cl)) / length(cl)
acc

[1] 0

In [14]:
#test
#x = list(list(1,2), list(3,4), list(5,6))
#a=do.call( rbind, x)[,1]
#as.numeric(a)

[1] 1 3 5